### Information Retrieval   |    Assignment 1-B

Step 1:

In [4]:
import pandas as pd
import numpy as np
import sys
import os

In [43]:
df_raw = pd.read_table('YandexRelPredChallenge.txt', delimiter="\t",index_col=None, 
                 header=0,names=['SessionID', 'TimePassed','TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID',
                                 '1','2','3','4','5',"6",'7','8'])
df_raw

In [95]:
# Open file    
fileHandler = open ('YandexRelPredChallenge.txt', "r")
# Get list of all lines in file
listOfLines = fileHandler.readlines()
# Close file 
fileHandler.close()
#Empty frame to fill in
index=[x for x in range(0,100000)]
df = pd.DataFrame(index=index, columns=['SessionID', 'TimePassed',
                 'TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID'])
df.name='data'

# Iterate over the lines and fill in the dataframe
for ind, line in zip(index, listOfLines):
#     print('index')
    line = line.strip().split()
    if line[2]=='Q':

        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':line[3],'RegionID':line[4], 'ListOfURLs':line[5:], 'URLID':np.nan})
    else:
        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':np.nan,'RegionID':np.nan, 'ListOfURLs':np.nan, 'URLID':line[3:]})

In [100]:
# Getting the types right. 
df.SessionID = df.SessionID.astype(int)
df.TimePassed = df.TimePassed.astype(int)
df.TypeOfAction = df.TypeOfAction.astype(str)
df.RegionID = df.RegionID.astype(float)
df.TimePassed = df.TimePassed.astype(float)
def parse_ulrid(x):
    if type(x)==list:
        return x[0]
    else:
        return x
df.URLID = df.URLID.apply(parse_ulrid)

Inspect

In [182]:
df

,SessionID,TimePassed,TypeOfAction,QueryID,RegionID,ListOfURLs,URLID,rank_e,rank_p
0,0,0.0,Q,8,0.0,"[7, 103, 51, 92, 43, 12, 73, 69, 27, 105]",NaN,"[0, 1, 1]","[0, 1, 1]"
1,0,36.0,Q,174,0.0,"[1625, 1627, 1623, 1626, 1624, 1622, 1619, 162...",NaN,"[1, 1, 1]","[0, 1, 1]"
2,0,50.0,Q,227,0.0,"[2094, 2091, 2087, 2089, 2093, 2088, 2090, 209...",NaN,"[0, 1, 0]","[0, 1, 0]"
3,0,515.0,Q,174,0.0,"[1625, 1627, 1623, 1626, 1624, 1622, 1619, 162...",NaN,"[1, 1, 1]","[0, 1, 1]"
4,0,524.0,Q,1974,0.0,"[17562, 1627, 1626, 1623, 2091, 17559, 17563, ...",NaN,"[1, 1, 0]","[0, 0, 0]"
5,0,527.0,C,NaN,NaN,NaN,17562,NaN,NaN
6,0,528.0,C,NaN,NaN,NaN,1627,NaN,NaN
7,0,529.0,C,NaN,NaN,NaN,1626,NaN,NaN
8,1,0.0,Q,9,0.0,"[13, 70, 66, 94, 50, 104, 29, 21, 89, 85]",NaN,"[0, 1, 1]","[1, 0, 0]"
9,1,20.0,C,NaN,NaN,NaN,104,NaN,NaN


In [116]:
test = df[df.SessionID==0]

So lets say that our experimental system is what Yandex return as a list now. Production or the system that is going to 'suck' (sort of a dumb system that we are going to compare against) will be constructed as shuffling each ranking for a query. 

Now, other approach, probably the better one: take the initial order as the 'benchmark' create only list of 0,1 (relevnace scores) for each system.
    - rember that we are interested only in three [3] first results.

In [214]:
# Create the relevance score ranking (those tables = that we called 'runs')
def relevance_list(l,acc=0.5, res_length=3):
    """ Control the 'accuracy'/'performance' with acc param. Only lists of length 3. """
    if type(l)==list:
        res = list()
        for x in l:
            if rnd.random()<acc:
                res.append((x,1))
            else:
                res.append((x,0))
        return rnd.sample(res, res_length)
    else:
        return l

df['rank_id_rel_e'] = df.ListOfURLs.apply(relevance_list, acc=0.7) 
df['rank_id_rel_p'] = df.ListOfURLs.apply(relevance_list) 

In [215]:
def extract_rel(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[1])
        return res
    else:
        return l
def extract_id(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[0])
        return res
    else:
        return l
df['rank_rel_e'] = df.rank_id_rel_e.apply(extract_rel)
df['rand_rel_p'] = df.rank_id_rel_p.apply(extract_rel)
df['rank_id_e'] = df.rank_id_rel_e.apply(extract_id)
df['rand_id_p'] = df.rank_id_rel_p.apply(extract_id)

In [216]:
df

,SessionID,TimePassed,TypeOfAction,QueryID,RegionID,ListOfURLs,URLID,rank_id_rel_e,rank_id_rel_p,rank_rel_e,rand_rel_p,rank_id_e,rand_id_p
0,0,0.0,Q,8,0.0,"[7, 103, 51, 92, 43, 12, 73, 69, 27, 105]",NaN,"[(27, 1), (7, 0), (69, 1)]","[(7, 1), (51, 0), (27, 0)]","[1, 0, 1]","[1, 0, 0]","[27, 7, 69]","[7, 51, 27]"
1,0,36.0,Q,174,0.0,"[1625, 1627, 1623, 1626, 1624, 1622, 1619, 162...",NaN,"[(1625, 1), (1626, 1), (1620, 1)]","[(1621, 0), (1624, 0), (1627, 0)]","[1, 1, 1]","[0, 0, 0]","[1625, 1626, 1620]","[1621, 1624, 1627]"
2,0,50.0,Q,227,0.0,"[2094, 2091, 2087, 2089, 2093, 2088, 2090, 209...",NaN,"[(2088, 1), (2095, 1), (2094, 0)]","[(2095, 0), (2089, 0), (2091, 0)]","[1, 1, 0]","[0, 0, 0]","[2088, 2095, 2094]","[2095, 2089, 2091]"
3,0,515.0,Q,174,0.0,"[1625, 1627, 1623, 1626, 1624, 1622, 1619, 162...",NaN,"[(1618, 1), (1620, 1), (1619, 1)]","[(1625, 1), (1626, 0), (1624, 0)]","[1, 1, 1]","[1, 0, 0]","[1618, 1620, 1619]","[1625, 1626, 1624]"
4,0,524.0,Q,1974,0.0,"[17562, 1627, 1626, 1623, 2091, 17559, 17563, ...",NaN,"[(1627, 1), (17559, 1), (17563, 0)]","[(1627, 0), (17558, 1), (2091, 1)]","[1, 1, 0]","[0, 1, 1]","[1627, 17559, 17563]","[1627, 17558, 2091]"
5,0,527.0,C,NaN,NaN,NaN,17562,NaN,NaN,NaN,NaN,NaN,NaN
6,0,528.0,C,NaN,NaN,NaN,1627,NaN,NaN,NaN,NaN,NaN,NaN
7,0,529.0,C,NaN,NaN,NaN,1626,NaN,NaN,NaN,NaN,NaN,NaN
8,1,0.0,Q,9,0.0,"[13, 70, 66, 94, 50, 104, 29, 21, 89, 85]",NaN,"[(21, 1), (94, 1), (29, 0)]","[(89, 1), (85, 1), (70, 1)]","[1, 1, 0]","[1, 1, 1]","[21, 94, 29]","[89, 85, 70]"
9,1,20.0,C,NaN,NaN,NaN,104,NaN,NaN,NaN,NaN,NaN,NaN


Step 2: Calculate ERR

In [213]:
# l = [(1,1),(2,2),(55,4)]
# print( extract_id(l) )
# print( extract_rel(l))

[1, 2, 55]
[1, 2, 4]


In [ ]:
%%latex
Calculate ERR for each query:

\begin{equation}
 ERR &= \sum_{r=1}^{N}( \prod_{i=1}^{r-1}(1-\theta_i)\theta_r)\frac{1}{r} 
\end{equation}
    
\begin{equation}
    \theta_i &= \frac{2^{rel} -1 }{2^{max_rel}}
\end{equation}

In [ ]:
def ERR(l):
    """ List or relevance scores. """ 
    if type(l)==list:
        # calculate the thetas first
        theta_list = [ (2**i -1)/(2**1) for i in l]
        
        # Calcualate total ERR score for the ranking 
    
        return res
    else:
        return l

### QUESTOINS 
    -how do we do the step one